### Timing to determing if process is cpu or i/o bound
##### Results indicate current code is CPU limited

In [2]:
%load_ext line_profiler

In [1]:
import os
import pandas as pd
import scipy.io as sio
import numpy as np
from scipy.sparse import coo_matrix
from numba.decorators import jit
from scipy.sparse.linalg import eigs

In [3]:
_parameterFile_ = os.path.abspath('/Users/camerongallivan/Research/Hybrid_PBN_Project/Compute_Rate_Matrix_Python/outputs_tmp/Trial_0001-py/paramValues.csv')
_outputPath_ = os.path.abspath('/Users/camerongallivan/Research/Hybrid_PBN_Project/Compute_Rate_Matrix_Python/outputs_tmp/Trial_0001-py')

In [13]:
#%lprun -f SimulationWrapper SimulationWrapper(_parameterFile_, _outputPath_)
%timeit SimulationWrapper(_parameterFile_, _outputPath_)

<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
<class 'scipy.sparse.csc.csc_matrix'>
1.43 s ± 53.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Modified SimulationWrapper.py

In [12]:
def SimulationWrapper(_parameterFile, _outputPath):
    parametersDF = pd.read_csv(_parameterFile, index_col=0)
    outputPath = os.path.abspath(_outputPath)

    rateMatrixPath = os.path.join(outputPath, 'RateMatrix')
    eigenValuesPath =  os.path.join(outputPath, 'EigenValues')
    probVecPath =  os.path.join(outputPath, 'ProbVec')
    prob2DPath = os.path.join(outputPath, 'Prob2D')
    timeScalesPath = os.path.join(outputPath, 'TimeScales')

    for row in parametersDF.itertuples():
        #print(row)
        saveFileName = 'set_{:05}.mat'.format(row.Index)

        # Calculating and saving rate matrix
        RateMatrix, Dimensions, StatesDict = main(row)
        print(type(RateMatrix))
        sio.savemat(os.path.join(rateMatrixPath, saveFileName), {'RateMatrix': RateMatrix, 'Dimensions': Dimensions}, do_compression=True)

        # Calculating and saving ProbVec, Prob2D and eigenvalues
        eigenValues, probVec, prob2D, timeScales = calc_probvec_prob2d(RateMatrix, Dimensions)
        sio.savemat(os.path.join(eigenValuesPath, saveFileName), {'EigenValues': eigenValues})
        sio.savemat(os.path.join(probVecPath, saveFileName), {'ProbVec': probVec})
        sio.savemat(os.path.join(prob2DPath, saveFileName), {'Prob2d': prob2D})
        sio.savemat(os.path.join(timeScalesPath, saveFileName), {'TimeScales': timeScales})

    np.save(os.path.join(outputPath, 'StatesDict.npy'), StatesDict)


### Compute_RateMatrix_MISAEx.py

In [6]:
def MISA_Ex_Rxn(parameters):
    # Parameters and model name
    model = 'MISAEx'
    paramSetNum = parameters.Index
    N = parameters.N
    g0 = parameters.g0
    g1 = parameters.g1
    kd = parameters.kd
    ha = parameters.ha
    hr = parameters.hr
    fa = parameters.fa
    fr = parameters.fr
    model_name = model + '_N{}'.format(N)

    # Copy number lists
    A = list(range(N+1))
    B = list(range(N+1))

    # Defining Gene States
    GeneA_00, GeneA_01, GeneA_10 = [0,1], [0,1], [0,1]
    GeneB_00, GeneB_01, GeneB_10 = [0,1], [0,1], [0,1]
    GeneA_States=[[1,0,0], [0,1,0], [0,0,1]]
    GeneB_States=[[1,0,0], [0,1,0], [0,0,1]]

    # Number of Microstate = CopyA * CopyB * GeneStatesA * GeneStatesB
    NumStates = (N+1) * (N+1) * 3 * 3

    # Initializing Rxn dict to hold Reactions, Species and Parameters
    Rxn = {}
    NumRxn = 16
    NumSpec = 8

    Rxn['Parameters'] = np.array([g0,g1,g0,ha,hr,fa,fr,kd,g0,g1,g0,ha,hr,fa,fr,kd])
    Rxn['Law'] = np.zeros((NumRxn, NumSpec), dtype=int)
    Rxn['Stoich'] = np.zeros((NumRxn, NumSpec), dtype=int)

    # Reaction Rate Laws, number of each species involved in the reaction
    Rxn['Law'][0,2]=1
    Rxn['Law'][1,3]=1
    Rxn['Law'][2,4]=1
    Rxn['Law'][3,0]=2
    Rxn['Law'][3,2]=1
    Rxn['Law'][4,1]=2
    Rxn['Law'][4,2]=1
    Rxn['Law'][5,3]=1
    Rxn['Law'][6,4]=1
    Rxn['Law'][7,0]=1

    Rxn['Law'][8,5]=1
    Rxn['Law'][9,6]=1
    Rxn['Law'][10,7]=1
    Rxn['Law'][11,1]=2
    Rxn['Law'][11,5]=1
    Rxn['Law'][12,0]=2
    Rxn['Law'][12,5]=1
    Rxn['Law'][13,6]=1
    Rxn['Law'][14,7]=1
    Rxn['Law'][15,1]=1

    # Reaction Stoichiometry, change in species resulting from reaction
    Rxn['Stoich'][0,0] = 1
    Rxn['Stoich'][1,0] = 1
    Rxn['Stoich'][2,0] = 1
    Rxn['Stoich'][3,0] = -2
    Rxn['Stoich'][4,1] = -2
    Rxn['Stoich'][3,2] = -1
    Rxn['Stoich'][3,3] = 1
    Rxn['Stoich'][4,2] = -1
    Rxn['Stoich'][4,4] = 1
    Rxn['Stoich'][5,0] = 2
    Rxn['Stoich'][6,1] = 2
    Rxn['Stoich'][5,3] = -1
    Rxn['Stoich'][5,2] = 1
    Rxn['Stoich'][6,4] = -1
    Rxn['Stoich'][6,2] = 1
    Rxn['Stoich'][7,0] = -1

    Rxn['Stoich'][8,1] = 1
    Rxn['Stoich'][9,1] = 1
    Rxn['Stoich'][10,1] = 1
    Rxn['Stoich'][11,1] = -2
    Rxn['Stoich'][12,0] = -2
    Rxn['Stoich'][11,5] = -1
    Rxn['Stoich'][11,6] = 1
    Rxn['Stoich'][12,5] = -1
    Rxn['Stoich'][12,7] = 1
    Rxn['Stoich'][13,1] = 2
    Rxn['Stoich'][14,0] = 2
    Rxn['Stoich'][13,6] = -1
    Rxn['Stoich'][13,5] = 1
    Rxn['Stoich'][14,7] = -1
    Rxn['Stoich'][14,5] = 1
    Rxn['Stoich'][15,1] = -1

    return Rxn, A, B, NumStates, NumSpec, NumRxn, GeneA_States, GeneB_States


def Determine_StatesDict(Dimensions, GeneA_States, GeneB_States):
    # original stateslist creation, returns ordered numpy array
    States_dict = {}

    for i in range(Dimensions[0]):
        for j in range(Dimensions[1]):
            for k in range(Dimensions[2]):
                for l in range(Dimensions[3]):
                    Cur = (i, j, *GeneA_States[k], *GeneB_States[l])
                    CurInd = np.ravel_multi_index((i, j, k, l), Dimensions, order='F')
                    States_dict[Cur] = CurInd
    return States_dict


def Calc_RateMatrix( Rxn, StatesDict, NumStates, NumRxn):
    StatesKeys = set(StatesDict.keys())
    MaxNumInteractions = NumStates * NumRxn
    CurInd_vals = np.zeros((MaxNumInteractions))
    DestInd_vals = np.zeros((MaxNumInteractions))
    RateMatrix_vals = np.zeros((MaxNumInteractions))

    n=0
    for state in StatesKeys:
        Cur = np.array(state, dtype=int)
        CurInd = StatesDict[state]
        for m in range(NumRxn):
            TestDest = tuple(Cur + Rxn['Stoich'][m,:])
            if TestDest in StatesKeys:
                n=n+1
                DestInd = StatesDict[TestDest]
                CurInd_vals, DestInd_vals, RateMatrix_vals = Update_RateMatrix(CurInd_vals, DestInd_vals, RateMatrix_vals, Cur, CurInd, DestInd, Rxn['Parameters'], Rxn['Law'], m, n)

    RateMatrix = coo_matrix((RateMatrix_vals, (DestInd_vals, CurInd_vals)), shape=(NumStates, NumStates)).tolil()
    RateMatrix.setdiag((RateMatrix.diagonal() - RateMatrix.sum(axis=0)).A[0])
    RateMatrix = RateMatrix.tocsc()
    
    return RateMatrix

def main(inputs):
    Rxn, A, B, NumStates, NumSpec, NumRxn, GeneA_States, GeneB_States = MISA_Ex_Rxn(inputs)
    Dimensions = [len(A), len(B), len(GeneA_States), len(GeneB_States)]
    StatesDict = Determine_StatesDict(Dimensions, GeneA_States, GeneB_States)
    RateMatrix = Calc_RateMatrix( Rxn, StatesDict, NumStates, NumRxn)

    return RateMatrix, Dimensions, StatesDict


### common_calcs.py

In [7]:
@jit('Tuple((f8[:], f8[:], f8[:]))(f8[:], f8[:], f8[:], i8[:], i8, i8, f8[:], i8[:,:], i8, i8)')
def Update_RateMatrix(CurInd_vals, DestInd_vals, RateMatrix_vals, Cur, CurInd, DestInd, parameters, laws, m, n):
    par = parameters[m]
    law = laws[m,:]
    for i in range(Cur.shape[0]):
        tmp1 = Cur[i]**law[i]
        tmp2 = 1
        for j in range(law[i]):
            tmp2 = tmp2 * (j+1)
        par = par * (tmp1/tmp2)

    CurInd_vals[n] = CurInd
    DestInd_vals[n] = DestInd
    RateMatrix_vals[n] = par

    return CurInd_vals, DestInd_vals, RateMatrix_vals

### RateMatrix_Calcs.py

In [9]:
def calc_probvec_prob2d(matrix, dimensions, dims_to_reduce=(2,3), nsig=15, sigma=1e-12):
    eigen_values, right_eigen_vectors = eigs(matrix, k=nsig, sigma=sigma)
    prob_vec = right_eigen_vectors[:,1]/np.sum(right_eigen_vectors[:,1])

    prob_full_d = prob_vec.reshape(dimensions, order='F')
    prob_2d = np.sum(prob_full_d, axis=dims_to_reduce)

    time_scales = -np.divide(1, eigen_values[1:].real)

    return eigen_values, prob_vec, prob_2d, time_scales

##### Timing results for converting lil-matrix to different sparse type before calculating eigs, used to be seperate cells

In [7]:
print('none')
%timeit SimulationWrapper(_parameterFile_, _outputPath_)
# 4.03 s ± 233 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

print('csc')
%timeit SimulationWrapper(_parameterFile_, _outputPath_)

# 3.73 s ± 73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

print('csr')
%timeit SimulationWrapper(_parameterFile_, _outputPath_)

# 3.84 s ± 85.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)